In [1]:
# !git clone https://github.com/anubhavmaity/Sports-Type-Classifier.git

In [2]:
# apt-get install p7zip
# wget https://datameetgeobk.s3.amazonaws.com/mldata/sports-type-classifier-data.7z
# p7zip -d  sports-type-classifier-data.7z

In [3]:
from fastai.vision import *

In [4]:
path = Path('data/')
# dest = path/folder
# dest.mkdir(parents=True, exist_ok=True)

In [5]:
path.ls()[:2]

[PosixPath('data/wwe_urls.txt'), PosixPath('data/weight_lifting_urls.txt')]

In [6]:
classes = ['badminton','baseball','basketball', 'boxing', 'chess', 'cricket', 'fencing', 'football', 'formula1', 'gymnastics', 'hockey', 'ice_hockey', 'kabaddi', 'motogp', 'shooting', 'swimming', 'table_tennis', 'tennis', 'volleyball', 'weight_lifting', 'wrestling', 'wwe']

In [7]:
# classes = ['ice_hockey', 'hockey', 'motogp']

In [8]:
len(classes)

22

In [9]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [10]:
# If you already cleaned your data, run this cell instead of the one before
# np.random.seed(42)
# data = ImageDataBunch.from_csv(".", folder=".", valid_pct=0.2, csv_labels='cleaned.csv',
#         ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [11]:
data.classes

['badminton',
 'baseball',
 'basketball',
 'boxing',
 'chess',
 'cricket',
 'fencing',
 'football',
 'formula1',
 'gymnastics',
 'hockey',
 'ice_hockey',
 'kabaddi',
 'motogp',
 'shooting',
 'swimming',
 'table_tennis',
 'tennis',
 'volleyball',
 'weight_lifting',
 'wrestling',
 'wwe']

In [12]:
len(data.classes)

22

In [14]:
data.show_batch(rows=1, figsize=(9,10))

RuntimeError: DataLoader worker (pid(s) 796) exited unexpectedly

In [15]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

(['badminton',
  'baseball',
  'basketball',
  'boxing',
  'chess',
  'cricket',
  'fencing',
  'football',
  'formula1',
  'gymnastics',
  'hockey',
  'ice_hockey',
  'kabaddi',
  'motogp',
  'shooting',
  'swimming',
  'table_tennis',
  'tennis',
  'volleyball',
  'weight_lifting',
  'wrestling',
  'wwe'],
 22,
 11524,
 2881)

## Train model

In [16]:
learn = cnn_learner(data, models.resnet34, metrics=[error_rate, accuracy])

In [17]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


RuntimeError: DataLoader worker (pid(s) 803) exited unexpectedly

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 1e-02

In [ ]:
learn.callback_fns.append(partial(SaveModelCallback, name='stage-1'))

In [ ]:
learn.fit_one_cycle(10, slice(lr))

In [ ]:
learn.load('stage-1');

In [ ]:
learn.unfreeze()

In [ ]:
learn.callback_fns.pop()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.callback_fns.append(partial(SaveModelCallback, name='unfreeze-stage-1'))

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-05, lr/5))

## Going towards a bigger model

In [ ]:
arch = models.resnet50

In [ ]:
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=299, num_workers=4, bs=32).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, arch, metrics=[error_rate, accuracy])

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.callback_fns.append(partial(SaveModelCallback, name='res50-stage-1'))

In [ ]:
lr = 1e-02

In [ ]:
learn.fit_one_cycle(5, slice(1e-02))

In [ ]:
learn.load('res50-stage-1');

In [ ]:
learn.callback_fns.pop()

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(skip_end=2)

In [ ]:
learn.callback_fns.append(partial(SaveModelCallback, name='res50-stage-2'))

In [ ]:
learn.fit_one_cycle(5, slice(1e-05, lr/5))

In [ ]:
learn.callback_fns.pop()

In [ ]:
learn.callback_fns.append(partial(SaveModelCallback, name='res50-stage-3'))

In [ ]:
learn.load('res50-stage-2');

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(5, slice(1e-05, lr/5))

In [ ]:
learn.load('res50-stage-3');

In [ ]:
learn.callback_fns.pop()

In [ ]:
learn.callback_fns.append(partial(SaveModelCallback, name='res50-stage-4'))

In [ ]:
learn.fit_one_cycle(7, slice(1e-05, lr/5))

## Model Summary

In [ ]:
learn.load('res50-stage-3');

In [ ]:
learn.unfreeze()

In [ ]:
print(learn.summary())

## Data Augmentation

In [ ]:
def _plot(i,j,ax):
    x,y = data.train_ds[3]
    x.show(ax, y=y)

plot_multi(_plot, 3, 3, figsize=(8,8))

## Interpretation

In [ ]:
learn.load('res50-stage-3');

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
interp.plot_top_losses(12, figsize=(15,11))

In [ ]:
## Train the model on bigger

## Heatmaps

In [ ]:
x,y = data.valid_ds[4]

In [ ]:
x.show()

In [ ]:
learn.predict(x)

In [ ]:
m = learn.model.eval()

In [ ]:
xb,_ = data.one_item(x)
xb_im = Image(data.denorm(xb)[0])
xb = xb.cuda()

In [ ]:
from fastai.callbacks.hooks import *

In [ ]:
def hooked_backward(cat=y):
    with hook_output(m[0]) as hook_a: 
        with hook_output(m[0], grad=True) as hook_g:
            preds = m(xb)
            preds[0,int(cat)].backward()
    return hook_a,hook_g

In [ ]:
hook_a,hook_g = hooked_backward()

In [ ]:
acts  = hook_a.stored[0].cpu()
acts.shape

In [ ]:
avg_acts = acts.mean(0)
avg_acts.shape

In [ ]:
def show_heatmap(hm):
    _,ax = plt.subplots()
    xb_im.show(ax)
    ax.imshow(hm, alpha=0.6, extent=(0,299,299,0),
              interpolation='bilinear', cmap='magma');

In [ ]:
show_heatmap(avg_acts)